In [541]:
import os
import arff
import numpy as np
import pandas as pd
import mlflow

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, FunctionTransformer, KBinsDiscretizer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import TweedieRegressor

from xgboost import XGBRegressor



In [542]:
RANDOM_SEED = 42


## Data Exploration

In [543]:
data_dir = './data'

#   Part 1
data_freq = arff.load(os.path.join(data_dir, 'freMTPL2freq.arff'))

df_freq = pd.DataFrame(data_freq , columns=["IDpol", "ClaimNb", "Exposure", "Area", "VehPower",
"VehAge", "DrivAge", "BonusMalus", "VehBrand", "VehGas", "Density", "Region"] )


display(df_freq.head())

print("Dataset size: ", len(df_freq))

print("IDpol has only unique values? ", (df_freq['IDpol'].nunique() == len(df_freq['IDpol'])))

display('Description of dataset: ', df_freq.describe(include='all'))



,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1.0,1.0,0.10,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82'
1,3.0,1.0,0.77,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82'
2,5.0,1.0,0.75,'B',6.0,2.0,52.0,50.0,'B12',Diesel,54.0,'R22'
3,10.0,1.0,0.09,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72'
4,11.0,1.0,0.84,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72'


Dataset size:  678013
IDpol has only unique values?  True


'Description of dataset: '

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
count,6.780130e+05,678013.000000,678013.000000,678013,678013.000000,678013.000000,678013.000000,678013.000000,678013,678013,678013.000000,678013
unique,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,11,2,NaN,22
top,NaN,NaN,NaN,'C',NaN,NaN,NaN,NaN,'B12',Regular,NaN,'R24'
freq,NaN,NaN,NaN,191880,NaN,NaN,NaN,NaN,166024,345877,NaN,160601
mean,2.621857e+06,0.053247,0.528750,NaN,6.454631,7.044265,45.499122,59.761502,NaN,NaN,1792.422405,NaN
std,1.641783e+06,0.240117,0.364442,NaN,2.050906,5.666232,14.137444,15.636658,NaN,NaN,3958.646564,NaN
min,1.000000e+00,0.000000,0.002732,NaN,4.000000,0.000000,18.000000,50.000000,NaN,NaN,1.000000,NaN
25%,1.157951e+06,0.000000,0.180000,NaN,5.000000,2.000000,34.000000,50.000000,NaN,NaN,92.000000,NaN
50%,2.272152e+06,0.000000,0.490000,NaN,6.000000,6.000000,44.000000,50.000000,NaN,NaN,393.000000,NaN
75%,4.046274e+06,0.000000,0.990000,NaN,7.000000,11.000000,55.000000,64.000000,NaN,NaN,1658.000000,NaN


In [544]:

#   Part 2

data_sev = arff.load(os.path.join(data_dir, 'freMTPL2sev.arff'))

df_sev = pd.DataFrame(data_sev, columns=["IDpol", "ClaimAmount"])


display(df_sev.head())

print("Dataset size: ", len(df_sev))

print("Percentage of IDpol-contracts with only one ClaimAmount: ", 
      round(df_sev['IDpol'].nunique()/len(df_sev['IDpol']), 4))

display('Description of dataset: ', df_sev.describe(include='all'))


print('Data types in ClaimAmount column:', df_sev["ClaimAmount"].apply(type).unique(), '\n')



,IDpol,ClaimAmount
0,1552.0,995.20
1,1010996.0,1128.12
2,4024277.0,1851.11
3,4007252.0,1204.00
4,4046424.0,1204.00


Dataset size:  26639
Percentage of IDpol-contracts with only one ClaimAmount:  0.9366


'Description of dataset: '

,IDpol,ClaimAmount
count,2.663900e+04,2.663900e+04
mean,2.279864e+06,2.278536e+03
std,1.577202e+06,2.929748e+04
min,1.390000e+02,1.000000e+00
25%,1.087642e+06,6.868100e+02
50%,2.137413e+06,1.172000e+03
75%,3.180162e+06,1.228080e+03
max,6.113971e+06,4.075401e+06


Data types in ClaimAmount column: [<class 'float'>] 



## Data preprocessing

### Create Features and Labels Dataframe

In [545]:
#   Sum up ClaimAmount for each distinct IDpol (contract)

df_sev_sum = df_sev.groupby(['IDpol'])[['ClaimAmount']].agg('sum').reset_index()

display(df_sev_sum)

,IDpol,ClaimAmount
0,139.0,303.00
1,190.0,1981.84
2,414.0,1456.55
3,424.0,10834.00
4,463.0,3986.67
...,...,...
24945,6113521.0,1324.40
24946,6113793.0,1769.88
24947,6113817.0,1288.28
24948,6113834.0,12230.40


In [546]:
#   Left join features table with ClaimAmount-table (containing unique IDpol)

df_combined = df_freq.merge(df_sev_sum, on="IDpol", how='left')

#   Fill None values of ClaimAmount with 0.0
df_combined["ClaimAmount"] = df_combined["ClaimAmount"].fillna(value=0.0)


#     Correct ClaimNb to 0 if ClaimAmount is 0 (assuming that ClaimAmount is ground truth)
df_combined["ClaimNb"] = np.where(df_combined["ClaimAmount"] == 0, 0, df_combined["ClaimNb"])


#   Construct the Pure Premium column (ClaimAmount / Exposure)
df_combined["ClaimAmount_Per_Year"] = df_combined["ClaimAmount"] / df_combined["Exposure"]

display(df_combined)

# TODO: drop 5% outliers w.r.t. certain columns?


print("Percentage of Policy Holders with no claims: ", 
      round(len(df_combined[df_combined["ClaimNb"] == 0.0]) / len(df_combined), 4))


,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,ClaimAmount_Per_Year
0,1.0,0.0,0.10000,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82',0.0,0.0
1,3.0,0.0,0.77000,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82',0.0,0.0
2,5.0,0.0,0.75000,'B',6.0,2.0,52.0,50.0,'B12',Diesel,54.0,'R22',0.0,0.0
3,10.0,0.0,0.09000,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72',0.0,0.0
4,11.0,0.0,0.84000,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72',0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678008,6114326.0,0.0,0.00274,'E',4.0,0.0,54.0,50.0,'B12',Regular,3317.0,'R93',0.0,0.0
678009,6114327.0,0.0,0.00274,'E',4.0,0.0,41.0,95.0,'B12',Regular,9850.0,'R11',0.0,0.0
678010,6114328.0,0.0,0.00274,'D',6.0,2.0,45.0,50.0,'B12',Diesel,1323.0,'R82',0.0,0.0
678011,6114329.0,0.0,0.00274,'B',4.0,0.0,60.0,50.0,'B12',Regular,95.0,'R26',0.0,0.0


Percentage of Policy Holders with no claims:  0.9632


In [547]:
#   Check if num of IDpols is correct

if len(df_combined) != len(df_freq):
    raise Exception("Wrong dimensions for df_combined")

In [548]:
#   (Optional) Randomly drop certain amount of No-Claim cases (vast majority)
#   so that balance between Claim <-> No-Claim cases is better

"""

no_claim_cases_indices = df_combined[df_combined["ClaimAmount"] == 0.0].index

#   Important to put here (cell re-run...)
np.random.seed(RANDOM_SEED)

indices_for_dropping = np.random.choice(no_claim_cases_indices, 
                                        size=200000,  # 625000
                                        replace=False)

df_combined.drop(index=indices_for_dropping, inplace=True)

"""

'\n\nno_claim_cases_indices = df_combined[df_combined["ClaimAmount"] == 0.0].index\n\n#   Important to put here (cell re-run...)\nnp.random.seed(RANDOM_SEED)\n\nindices_for_dropping = np.random.choice(no_claim_cases_indices, \n                                        size=200000,  # 625000\n                                        replace=False)\n\ndf_combined.drop(index=indices_for_dropping, inplace=True)\n\n'

In [549]:
#   Create features dataset and perform encoding

X = df_combined.drop(
    columns=["IDpol", "ClaimNb", "Exposure", "ClaimAmount", "ClaimAmount_Per_Year"]).reset_index(drop=True)

print("Original Features:")
display(X)


#   Binning, Encoding and Scaling of certain columns

log_scale_transformer = make_pipeline(
    FunctionTransformer(func=np.log), StandardScaler()
)

column_trans = ColumnTransformer(
    [
        (
            "binned_numeric",
            KBinsDiscretizer(n_bins=10, subsample=int(2e5), random_state=RANDOM_SEED),
            ["VehAge", "DrivAge"],
        ),
        (
            "onehot_categorical",
            OneHotEncoder(),
            ["VehBrand", "VehPower", "VehGas", "Region", "Area"],
        ),
        ("passthrough_numeric", "passthrough", ["BonusMalus"]),
        ("log_scaled_numeric", log_scale_transformer, ["Density"]),
    ]
)

X_encoded = column_trans.fit_transform(X).toarray()


print(f"Encoded Features; type {type(X_encoded)}, shape {X_encoded.shape} : ")
display(X_encoded)


#############################################################


#   Potential target cols - here ClaimAmount_Per_Year will be used

Y = df_combined[["ClaimNb", "Exposure", "ClaimAmount", "ClaimAmount_Per_Year"]]

print(Y)


Original Features:


,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82'
1,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82'
2,'B',6.0,2.0,52.0,50.0,'B12',Diesel,54.0,'R22'
3,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72'
4,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72'
...,...,...,...,...,...,...,...,...,...
678008,'E',4.0,0.0,54.0,50.0,'B12',Regular,3317.0,'R93'
678009,'E',4.0,0.0,41.0,95.0,'B12',Regular,9850.0,'R11'
678010,'D',6.0,2.0,45.0,50.0,'B12',Diesel,1323.0,'R82'
678011,'B',4.0,0.0,60.0,50.0,'B12',Regular,95.0,'R26'


Encoded Features; type <class 'numpy.ndarray'>, shape (678013, 75) : 


array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,
        50.        ,  0.60005456],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
        50.        ,  0.60005456],
       [ 0.        ,  0.        ,  1.        , ...,  0.        ,
        50.        , -1.06540391],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  0.        ,
        50.        ,  0.64470317],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
        50.        , -0.76339517],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        54.        , -0.96628176]])

        ClaimNb  Exposure  ClaimAmount  ClaimAmount_Per_Year
0           0.0   0.10000          0.0                   0.0
1           0.0   0.77000          0.0                   0.0
2           0.0   0.75000          0.0                   0.0
3           0.0   0.09000          0.0                   0.0
4           0.0   0.84000          0.0                   0.0
...         ...       ...          ...                   ...
678008      0.0   0.00274          0.0                   0.0
678009      0.0   0.00274          0.0                   0.0
678010      0.0   0.00274          0.0                   0.0
678011      0.0   0.00274          0.0                   0.0
678012      0.0   0.00274          0.0                   0.0

[678013 rows x 4 columns]


### Splitting data

In [550]:
X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, Y, 
                                                    test_size=0.2, 
                                                    shuffle=True,
                                                    random_state=RANDOM_SEED
                                                    )

print("Shape of X_train: ", X_train.shape, " Shape of Y_train: ", Y_train.shape)
print("Shape of X_test: ", X_test.shape, " Shape of Y_test: ", Y_test.shape)

Shape of X_train:  (542410, 75)  Shape of Y_train:  (542410, 4)
Shape of X_test:  (135603, 75)  Shape of Y_test:  (135603, 4)


## Create Model

In [551]:
# Gradient-Boosted Decision Trees; (Hist)GradientBoostingRegressor or RandomForestRegressor


param_grid_hgbr = {
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'max_iter': [100, 150, 200, 300, 500, 800, 1000],
    'max_depth': [3, 5, 7, 9],
    'min_samples_leaf': [4, 6, 10, 15, 20, 25, 30],
    'max_leaf_nodes': [10, 20, 30, 40]
}

#  Optimized Histogram-based GBR  
hgbr = HistGradientBoostingRegressor(random_state=RANDOM_SEED)

#   Using 5-fold cross-validation
random_search = RandomizedSearchCV(estimator=hgbr, 
                                   param_distributions=param_grid_hgbr, 
                                   n_iter=15, 
                                   cv=5, 
                                   random_state=RANDOM_SEED)

random_search.fit(X=X_train, 
                  y=Y_train["ClaimAmount_Per_Year"], 
                  sample_weight=Y_train["Exposure"])


model = random_search.best_estimator_

print("Best params: ", random_search.best_params_)



Best params:  {'min_samples_leaf': 20, 'max_leaf_nodes': 30, 'max_iter': 500, 'max_depth': 3, 'learning_rate': 0.01}


In [552]:
#   Performance on train data

train_preds = model.predict(X_train)

mae_train = mean_absolute_error(Y_train["ClaimAmount_Per_Year"], train_preds)
mape_train = mean_absolute_percentage_error(Y_train["ClaimAmount_Per_Year"], train_preds)

print("Mean Absolute Error (train):", mae_train)
print("Mean Absolute Percentage Error (train):", mape_train)

Mean Absolute Error (train): 527.7840551667564
Mean Absolute Percentage Error (train): 7.226983073577879e+17


In [553]:
#   Performance test data

test_preds = model.predict(X_test)

mae_test = mean_absolute_error(Y_test["ClaimAmount_Per_Year"], test_preds)
mape_test = mean_absolute_percentage_error(Y_test["ClaimAmount_Per_Year"], test_preds)

print("Mean Absolute Error (test):", mae_test)
print("Mean Absolute Percentage Error (test):", mape_test)


Mean Absolute Error (test): 573.2538839649907
Mean Absolute Percentage Error (test): 7.233428762529283e+17
